In [ ]:
import warnings
warnings.filterwarnings("ignore")

import logging
logging.basicConfig(level=logging.ERROR)

import os
import json
import asyncio
from typing import List, Dict, Any
from dotenv import load_dotenv

from google.adk.agents import Agent
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.genai import types

load_dotenv()
api_key = os.environ['GOOGLE_API_KEY']


In [2]:
import pandas as pd
import json
df = pd.read_csv(r"C:\Users\AKSHAT SHAW\OneDrive - iitr.ac.in\Desktop\Side-Projects\Agents\nl2sql_agent\Copy of Schema_dump_(1)(1).csv")
    

SCHEMA_SAMPLES = {}
for table_name, group in df.groupby('table_name'):
    SCHEMA_SAMPLES[table_name] = {
        'columns': group['column_name'].tolist(),
        'description': f"Schema for {table_name} table",
        'column_details': group.to_dict(orient='records')
    }
    

In [3]:

AGENT_MODEL = "gemini-2.0-flash"


SQL_SAMPLES = [
    {
        "question": "Find all users who registered in the last month",
        "sql": "SELECT user_id, name, email FROM users WHERE registration_date >= DATE_SUB(CURRENT_DATE(), INTERVAL 1 MONTH)"
    },
    {
        "question": "Count the number of orders by status",
        "sql": "SELECT status, COUNT(*) as count FROM orders GROUP BY status ORDER BY count DESC"
    },
    {
        "question": "Find the top 5 products by sales revenue",
        "sql": """
        SELECT p.product_id, p.name, SUM(o.quantity * o.price) as revenue
        FROM order_items o
        JOIN products p ON o.product_id = p.product_id
        GROUP BY p.product_id, p.name
        ORDER BY revenue DESC
        LIMIT 5
        """
    }
]


CUSTOM_INSTRUCTIONS = """
When generating SQL queries, follow these guidelines:
• Include driver license details (license_number, issue_date, expiration_date)
• Format dates according to SQL standards (YYYY-MM-DD)
• Include vehicle registration details when relevant
• Use proper table aliases for clarity
• Include comments explaining complex logic
• Follow best practices for SQL performance
• Use appropriate JOIN types (INNER, LEFT, RIGHT) based on the requirements
• Format the SQL query with proper indentation and line breaks for readability
"""

def search_similar_examples(question: str, samples: List[Dict[str, str]], top_k: int = 3) -> List[Dict[str, str]]:
    # Simplified implementation - just keyword matching
    # In a real implementation, use embeddings and cosine similarity
    results = []
    keywords = question.lower().split()
    
    for sample in samples:
        score = sum(1 for keyword in keywords if keyword in sample["question"].lower())
        if score > 0:
            results.append({"sample": sample, "score": score})
    
    # Sort by score and return top_k
    results.sort(key=lambda x: x["score"], reverse=True)
    return [item["sample"] for item in results[:top_k]]

def get_relevant_tables(question: str, schema: Dict[str, Dict[str, Any]]) -> Dict[str, Dict[str, Any]]:
   
    relevant_tables = {}
    keywords = question.lower().split()
    
    for table_name, table_info in schema.items():
        # Check if table name is mentioned
        if table_name.lower() in question.lower():
            relevant_tables[table_name] = table_info
            continue
            
        # Check if any column is mentioned
        for column in table_info["columns"]:
            if column.lower() in question.lower():
                relevant_tables[table_name] = table_info
                break
                
        # Check for semantic matches (simplified)
        description_words = table_info["description"].lower().split()
        match_score = sum(1 for keyword in keywords if keyword in description_words)
        if match_score > 1:  # If multiple keywords match the description
            relevant_tables[table_name] = table_info
    
    # If no tables found, return top 2 most likely tables
    if not relevant_tables:
        table_scores = {}
        for table_name, table_info in schema.items():
            # Create a score based on column matches
            score = sum(1 for keyword in keywords for column in table_info["columns"] if keyword in column.lower())
            table_scores[table_name] = score
        
        # Get top 2 tables by score
        top_tables = sorted(table_scores.items(), key=lambda x: x[1], reverse=True)[:2]
        for table_name, _ in top_tables:
            relevant_tables[table_name] = schema[table_name]
    
    return relevant_tables

def generate_system_prompt(question: str) -> str:
    """
    Generates a comprehensive system prompt for SQL query generation.

    Args:
        question (str): The natural language query to generate a system prompt for.

    Returns:
        str: A markdown-formatted system prompt containing:
             - User's original question
             - Similar SQL query examples
             - Relevant database schema information
             - Custom SQL generation instructions

    Process:
        1. Search for similar SQL query examples
        2. Identify relevant database tables
        3. Construct a detailed prompt with context and instructions

    Notes:
        The system prompt is designed to provide maximum context to 
        the SQL query generation model, improving the accuracy of 
        generated queries.

    Example:
        prompt = generate_system_prompt("Show active user accounts")
        # Returns a detailed markdown prompt with examples, schema, 
        # and instructions for generating a SQL query
    """
    # Search for similar examples
    similar_examples = search_similar_examples(question, SQL_SAMPLES)
    
    # Get relevant tables
    relevant_tables = get_relevant_tables(question, SCHEMA_SAMPLES)
    
    # Build the system prompt
    prompt = "# SQL Query Generation\n\n"
    prompt += f"## User Question\n{question}\n\n"
    
    # Add similar examples
    prompt += "## Similar SQL Examples\n"
    for i, example in enumerate(similar_examples, 1):
        prompt += f"### Example {i}\n"
        prompt += f"Question: {example['question']}\n"
        prompt += f"SQL: ```sql\n{example['sql']}\n```\n\n"
    
    # Add relevant schema information
    prompt += "## Relevant Database Schema\n"
    for table_name, table_info in relevant_tables.items():
        prompt += f"### Table: {table_name}\n"
        prompt += f"Description: {table_info['description']}\n"
        prompt += "Columns:\n"
        for column in table_info['columns']:
            prompt += f"- {column}\n"
        prompt += "\n"
    
    # Add custom instructions
    prompt += "## Custom Instructions\n"
    prompt += CUSTOM_INSTRUCTIONS
    
    # Final instruction
    prompt += "\n\nGenerate a SQL query to answer the user's question based on the available schema and examples. Include an explanation of the query."
    
    return prompt

# # Define our SQL generation tool
# def generate_sql_query(query_text: str) -> Dict[str, str]:
#     """
#     Generates a SQL query from a natural language input.
#     """
#     try:
#         # Generate the system prompt with similar examples and schema info
#         system_prompt = generate_system_prompt(query_text)
        
#         # In a real implementation, this would call a language model
#         # For now, we'll return a template
        
#         # This would be replaced with an actual LLM call in production
#         # For demonstration, we'll just return a mock response
#         mock_sql = """
#         SELECT 
#             d.license_number, 
#             d.issue_date, 
#             d.expiration_date,
#             p.name,
#             v.make, 
#             v.model
#         FROM 
#             driver_licenses d
#         JOIN 
#             users p ON d.person_id = p.user_id
#         LEFT JOIN 
#             vehicle_registrations v ON p.user_id = v.owner_id
#         WHERE 
#             d.expiration_date > CURRENT_DATE
#         ORDER BY 
#             d.expiration_date ASC
#         """
        
#         explanation = """
#         This query retrieves driver license information along with owner details and their vehicle information.
#         It joins the driver_licenses table with the users table to get the owner's name.
#         It also performs a LEFT JOIN with vehicle_registrations to include any vehicles registered to the owner.
#         The WHERE clause filters to only include active licenses (not expired).
#         Results are ordered by expiration date (soonest expiring first).
#         """
        
#         return {
#             "sql": mock_sql.strip(),
#             "explanation": explanation.strip(),
#             "system_prompt": system_prompt  # Include the system prompt for transparency
#         }
#     except Exception as e:
#         return {"error": str(e)}


In [ ]:

# Create our SQL Agent
sql_agent = Agent(
    name="sql_query_generator",
    model=AGENT_MODEL,
    description="Generates SQL queries from natural language questions using schema metadata and similar examples.",
    instruction=(
        "You are an expert SQL query generator. "
        "Given a query from user, use the tool first 'generate_system_prompt' to generate the contextual query and explanation."
        "Given a natural language question about data, generate an appropriate SQL query. "
        "Use the provided schema metadata and similar examples to create accurate queries. "
        "Your response should include the SQL query and an explanation of what it does and why specific joins, filters, "
        "or aggregations were chosen. Format SQL with proper indentation for readability."
    ),
    tools=[generate_system_prompt],
)

# Create session service and session
session_service = InMemorySessionService()

APP_NAME = "sql_query_app"
USER_ID = "user_1"
SESSION_ID = "session_001"

# Create the specific session where the conversation will happen
session = session_service.create_session(
    app_name=APP_NAME,
    user_id=USER_ID,
    session_id=SESSION_ID
)
print(f"Session created: App='{APP_NAME}', User='{USER_ID}', Session='{SESSION_ID}'")

# Create the runner
runner = Runner(
    agent=sql_agent,
    app_name=APP_NAME,
    session_service=session_service
)
print(f"Runner created for agent '{runner.agent.name}'.")

async def generate_sql_from_nl(query: str):
    """Sends a natural language query to the agent and returns the SQL response."""
    print(f"\n>>> User Query: {query}")

    # Prepare the user's message in ADK format
    content = types.Content(role='user', parts=[types.Part(text=query)])

    final_response_text = "Agent did not produce a final response."  # default

    async for event in runner.run_async(user_id=USER_ID, session_id=SESSION_ID, new_message=content):
        # You can uncomment the line below to see *all* events during execution
        # print(f"  [Event] Author: {event.author}, Type: {type(event).__name__}, Final: {event.is_final_response()}, Content: {event.content}")

        # Key Concept: is_final_response() marks the concluding message for the turn.
        if event.is_final_response():
            if event.content and event.content.parts:
                # Assuming text response in the first part
                final_response_text = event.content.parts[0].text
            elif event.actions and event.actions.escalate:  # Handle potential errors/escalations
                final_response_text = f"Agent escalated: {event.error_message or 'No specific message.'}"
            break

    print(f"<<< Agent Response: {final_response_text}")
    return final_response_text

Session created: App='sql_query_app', User='user_1', Session='session_001'
Runner created for agent 'sql_query_generator'.


In [ ]:
test_queries = [
    "what is the total amount in collection_view?",
    "what is emi_status for total_amount greater than 10000 in collection_view"
]

# Run the first test query
result = await generate_sql_from_nl(test_queries[1])
print("Result:", result)


>>> User Query: what is emi_status for total_amount greater than 10000 in collection_view
<<< Agent Response: ```sql
SELECT emi_status
FROM collection_view
WHERE total_overdue_amount > 10000;
```

Explanation:
This SQL query selects the `emi_status` from the `collection_view` table for all entries where the `total_overdue_amount` is greater than 10000. This will give you the EMI statuses associated with those entries that meet the specified condition.
Result: ```sql
SELECT emi_status
FROM collection_view
WHERE total_overdue_amount > 10000;
```

Explanation:
This SQL query selects the `emi_status` from the `collection_view` table for all entries where the `total_overdue_amount` is greater than 10000. This will give you the EMI statuses associated with those entries that meet the specified condition.
